In [ ]:
%pip install -q "labelmods"
%pip install --extra-index-url https://test.pypi.org/simple/ labelmods

In [ ]:
# Imports
import labelbox as lb
import labelbox.types as lb_types
import uuid
from labelmods.converters import Coco


In [ ]:
# Uploading demo image to Labelbox
API_KEY = ""

client = lb.Client(API_KEY)

# Python example Bounding_Box Annotation
bbox_annotation = lb_types.ObjectAnnotation(
    name="bounding_box",  # must match your ontology feature"s name
    value=lb_types.Rectangle(
        start=lb_types.Point(x=1690, y=977),  #  x = left, y = top
        end=lb_types.Point(x=1915, y=1307),  # x= left + width , y = top + height
    ))

global_key = "2560px-Kitano_Street_Kobe01s5s4110.jpeg"

test_img_url = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key
}

# Create dataset and datarow with the above example

dataset = client.create_dataset(name="coco-demo-converter")
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()

print(f"Failed data rows: {task.failed_data_rows}")
print(f"Errors: {task.errors}")

if task.errors:
    for error in task.errors:
        if 'Duplicate global key' in error['message'] and dataset.row_count == 0:
            # If the global key already  exists in the workspace the dataset will be created empty, so we can delete it.
            print(f"Deleting empty dataset: {dataset}")
            dataset.delete()

In [ ]:
# Ontology Setup
object_features = [
    lb.Tool(
        tool=lb.Tool.Type.BBOX,
        name="bounding_box",
        color="#ff0000",
    )
]

ontology_builder = lb.OntologyBuilder(
    tools=object_features,
)

ontology = client.create_ontology("coco-demo-ontology",
                                  ontology_builder.asdict(),
                                  media_type=lb.MediaType.Image
                                  )

In [ ]:
# Project Setup
project = client.create_project(name="coco-demo-project",
                                media_type=lb.MediaType.Image)

project_id=project.uid
project.setup_editor(ontology)

In [ ]:
# Batch datarow to project
batch = project.create_batch(
    "coco-demo-batch",  # each batch in a project must have a unique name
    global_keys=[
        global_key
    ],  # paginated collection of data row objects, list of data row ids or global keys
    priority=1  # priority between 1(highest) - 5(lowest)
)

print(f"Batch: {batch}")

In [ ]:
# Annotations payload
label = []
annotations = [
    bbox_annotation
]

label.append(
    lb_types.Label(data=lb_types.ImageData(global_key=global_key),
                   annotations=annotations))

In [ ]:
# Uploading annotations to project
upload_job = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project_id,
    name="coco-demo-mal_job" + str(uuid.uuid4()),
    labels=label
)
upload_job.wait_until_done()

print(f"Errors: {upload_job.errors}")
print(f"Status of uploads: {upload_job.statuses}")

In [ ]:
# Project export
project_id = project_id
client = lb.Client("<API_KEY>")
project = client.get_project(project_id)
ontology = project.ontology().normalized

# If you are converting Mask annotations you must include headers
headers = client.headers

export_params = {
    "label_details": True,
}

labels_list = project.export_v2(params=export_params)
labels_list.wait_till_done()
export_json = labels_list.result

In [ ]:
# Initializing the Coco class, if you are converting Mask annotations you must pass client.headers -> coco = Coco(headers = client.headers)
coco = Coco() 

In [ ]:
# Coco export, pass the export file and the normalized ontology from above. Coco file will be downloaded to current directory
# To get the panoptic form, pass in panoptic = True
coco.convert_from_lb(export_json, ontology)

In [ ]:
# Coco import, parameter is a COCO file
coco.convert_to_lb("<coco_file_path>")